In [28]:
import pandas as pd
import os

from datetime import datetime

In [3]:
#read the raw data
df = pd.read_csv('logon.csv', usecols = ['user', 'date', 'pc', 'activity'])

In [31]:
#necessary functions

#return just date
def split_date(datetime):
    return datetime.date()

#return just time
def split_time(datetime):
    return datetime.time()

#determine if actions are performed in working hours
def is_working_hours(timestamp):
    time = timestamp.time()
    
    start = datetime.strptime('09:00:00', '%H:%M:%S').time()
    end = datetime.strptime('17:00:00', '%H:%M:%S').time()
    
    # Check if it's a weekday (Monday: 0, Tuesday: 1, ..., Friday: 4)
    if timestamp.weekday() in range(0, 5):
        if (time >= start and time <= end): # Check if it's 9 AM or later
            return "Work Hours"
        return "Off Hours"
    else:
        return 'Weekend'  # It's not a weekday

In [32]:
#turn date into datetime
df['date'] = pd.to_datetime(df.date, format = '%m/%d/%Y %H:%M:%S')

#split dates and times
df['only_date'] = df['date'].apply(split_date)
df['only_time'] = df['date'].apply(split_time)

df['WorkingHours'] = df.date.apply(is_working_hours)

In [34]:
#group dataframe by date
grouped = df.groupby('only_date')

In [35]:
#hold the file names to make a log file later
file_names = []

#create separate csv's for each date
for date, group in grouped:
    current_file_name = f'logon-{date}.csv'
    group.to_csv(f'logon_data/{current_file_name}', index = False)
    file_names.append(current_file_name)

In [25]:
#create a log file with the file names to make access easier
with open('logon_data/logon_files.log', 'w') as file:
    for name in file_names:
        file.write(name + '\n')